In [1]:
import gcp.bigquery as bq
import gcp.storage as storage
import numpy as np
try:
   import cPickle as pickle
except:
   import pickle
EST_PICKLE_FILENAME = 'baseline_final_estimator.pkl'

# First feature HAS to be 'district_id' for MAPE calculation.
fields_str = """gap, district_id, sum_price, traffic_tj_level3, traffic_tj_level2, traffic_tj_level4, f23,
  f16_10,f17_3, f19, f5_3, f11_3, f24_2, f20_2, f13_4, f4, f4_5, f11_2, f25, f25_3, f7, f11_4,
  f23_3, f17_4, f8_4, f15_7, f4_17, f8_2, f4_10, f15_2, f11_6, f15, f17_5, f1_4, f24_1, f1,
  f16_11, f19_3, f23_6, f2_2, f11, f15_3, f8_3, f15_6, f6_1, f6_4, f8, f11_8, f25_8, f23_4,
  f17_2, f1_5, f4_9, f1_8, f4_11, f20_8, f2_5, f16, f25_9, f6_2, f14_3, f15_4, f4_7, f5_1, f5,
  f4_18, f14_8, f11_7, f3_2, f20_5, f17, f11_1, f4_14, f23_5, f1_7, traffic_tj_level1, f23_2,
  f20, f1_3, f13_8, f1_2, f8_1, f23_1, f11_5, f2_12, f4_1, f2_7, f22_1, f25_7
"""
fields = map(lambda x: x.strip(), fields_str.split(','))
features = fields[1:]

# Prepare Final Dataset

In [2]:
%%sql --module q_all_f

SELECT *
FROM [datalab-projects-1331:xjk_algo_comp_test.future_gaps_processed]
WHERE gap > 0 AND timeslot IN ('2016-01-22-46','2016-01-22-58','2016-01-22-70','2016-01-22-82',
    '2016-01-22-94','2016-01-22-106','2016-01-22-118','2016-01-22-130','2016-01-22-142',
    '2016-01-24-58','2016-01-24-70','2016-01-24-82','2016-01-24-94','2016-01-24-106',
    '2016-01-24-118','2016-01-24-130','2016-01-24-142','2016-01-26-46','2016-01-26-58',
    '2016-01-26-70','2016-01-26-82','2016-01-26-94','2016-01-26-106','2016-01-26-118',
    '2016-01-26-130','2016-01-26-142','2016-01-28-58','2016-01-28-70','2016-01-28-82',
    '2016-01-28-94','2016-01-28-106','2016-01-28-118','2016-01-28-130','2016-01-28-142',
    '2016-01-30-46','2016-01-30-58','2016-01-30-70','2016-01-30-82','2016-01-30-94',
    '2016-01-30-106','2016-01-30-118','2016-01-30-130','2016-01-30-142')
ORDER BY timeslot, district_id

# Final dataset - Used in final submission.

In [3]:
query_f = bq.Query(q_all_f)
tableresult_f = query_f.results()

all_data_f = np.zeros((tableresult_f.length, len(fields)))
print 'there are {} rows'.format(tableresult_f.length)
for rcounter, row in enumerate(tableresult_f):
  for fcounter, field in enumerate(fields):
    all_data_f[rcounter, fcounter] = row[field]
  if rcounter % 1000 == 0:
    print 'processed {} rows'.format(rcounter)
    
all_data_f[np.isnan(all_data_f)] = 0
data_final = all_data_f[:,1:]
targets_final = all_data_f[:,0]

there are 1802 rows
processed 0 rows
processed 1000 rows


# Run Prediction and Export CSV

In [4]:
final_est = pickle.load(open(EST_PICKLE_FILENAME, "r") )

In [5]:
%%sql --module q_districts

SELECT district_id FROM [datalab-projects-1331:xjk_algo_comp_test.districts] ORDER BY district_id

In [6]:
query_districts = bq.Query(q_districts)
tableresult_districts = query_districts.results()
districts = [d['district_id'] for d in tableresult_districts]

In [7]:
final_predictions = final_est.predict(data_final)

slots = ['2016-01-22-46','2016-01-22-58','2016-01-22-70','2016-01-22-82',
    '2016-01-22-94','2016-01-22-106','2016-01-22-118','2016-01-22-130','2016-01-22-142',
    '2016-01-24-58','2016-01-24-70','2016-01-24-82','2016-01-24-94','2016-01-24-106',
    '2016-01-24-118','2016-01-24-130','2016-01-24-142','2016-01-26-46','2016-01-26-58',
    '2016-01-26-70','2016-01-26-82','2016-01-26-94','2016-01-26-106','2016-01-26-118',
    '2016-01-26-130','2016-01-26-142','2016-01-28-58','2016-01-28-70','2016-01-28-82',
    '2016-01-28-94','2016-01-28-106','2016-01-28-118','2016-01-28-130','2016-01-28-142',
    '2016-01-30-46','2016-01-30-58','2016-01-30-70','2016-01-30-82','2016-01-30-94',
    '2016-01-30-106','2016-01-30-118','2016-01-30-130','2016-01-30-142']

result_dicts = []
print "Preparing results..."
for slot in slots:
  for district in districts:
#     pred = 0.0
#     try:
#       id = (index for (index, item) in enumerate(tableresult_f) if 
#             item['timeslot'] == slot and item['district_id'] == id).next()
#       pred = final_predictions[id]
#     except:
#       pass
    result_dicts.append({'key': '{}:{}'.format(district, slot), 'value': \
      '{},{},{}'.format(district, slot,0.0)})

print 'Replacing predictions...'
for (index, item) in enumerate(tableresult_f):
  id = (index for (r_id, r_item) in enumerate(result_dicts) if 
    r_item['key'] == '{}:{}'.format(item['district_id'], item['timeslot'])).next()
  result_dicts[id]['value'] = \
    '{},{},{}'.format(item['district_id'], item['timeslot'],final_predictions[index])
  if index % 200 == 0:
    print 'Done {}/{}...'.format(index, tableresult_f.length)
    
result = '\n'.join(map(lambda d: d['value'], result_dicts))
bucketname = 'datalab-projects-1331-datalab'
itempath = 'result/final_result.csv'
print 'Done, now writing to gs://{}/{}'.format(bucketname, itempath)
item = storage.Item(bucketname, itempath)
item.write_to(result, 'text/plain')

Preparing results...
Replacing predictions...
Done 0/1802...
Done 200/1802...
Done 400/1802...
Done 600/1802...
Done 800/1802...
Done 1000/1802...
Done 1200/1802...
Done 1400/1802...
Done 1600/1802...
Done 1800/1802...
Done, now writing to gs://datalab-projects-1331-datalab/result/final_result.csv


In [8]:
print(result)

1,2016-01-22-106,10.0
2,2016-01-22-106,1.0
3,2016-01-22-106,1.0
4,2016-01-22-106,29.0
7,2016-01-22-106,753.0
8,2016-01-22-106,241.0
9,2016-01-22-106,2.0
11,2016-01-22-106,1.0
12,2016-01-22-106,14.0
13,2016-01-22-106,1.0
14,2016-01-22-106,140.0
16,2016-01-22-106,3.0
17,2016-01-22-106,1.0
18,2016-01-22-106,8.0
19,2016-01-22-106,13.0
20,2016-01-22-106,24.0
21,2016-01-22-106,7.0
22,2016-01-22-106,8.0
23,2016-01-22-106,68.0
24,2016-01-22-106,4.0
25,2016-01-22-106,3.0
26,2016-01-22-106,69.0
27,2016-01-22-106,8.0
28,2016-01-22-106,30.0
29,2016-01-22-106,3.0
31,2016-01-22-106,12.0
32,2016-01-22-106,1.0
33,2016-01-22-106,2.0
35,2016-01-22-106,13.0
36,2016-01-22-106,2.0
37,2016-01-22-106,144.0
38,2016-01-22-106,3.0
41,2016-01-22-106,1.0
42,2016-01-22-106,5.0
43,2016-01-22-106,1.0
44,2016-01-22-106,1.0
46,2016-01-22-106,33.0
47,2016-01-22-106,3.0
48,2016-01-22-106,565.0
51,2016-01-22-106,910.0
53,2016-01-22-106,9.0
57,2016-01-22-106,2.0
62,2016-01-22-106,1.0
66,2016-01-22-106,1.0
1,2016-01-22-118